In [ ]:
# LIBRARIES AND DATA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
import math
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
warnings.filterwarnings('ignore')


## Functions

In [7]:
def clean(df):
    for i in range(len(df)):
        df['Crude Rate'].iloc[i] = df['Crude Rate'].iloc[i].replace(' (Unreliable)', '')
    
    x=['Notes','State Code','Year Code', 'Gender Code', 'Race Code', 'Age Group']
    for i in x:
        df = df.drop(i, axis=1)
    return df
    

In [8]:
def checkvol(df):
    x=['Gender', 'Race','Age Group Code']
    for i in x:
        print(df[i].unique())

In [9]:
def Unifiedformat(df,colname,colvol):
    x=df.loc[df[colname]==colvol].index
    df=df.drop(index=x)
    return df

In [10]:
def createData2(df,df2,df3,df4):
    df5=df
    df5['breath Deaths']=0
    df5['breath Population']=0
    df5['breath Crude Rate']=0
    
    df5['cancer Deaths']=0
    df5['cancer Population']=0
    df5['cancer Crude Rate']=0
    
    df5['total Deaths']=0
    df5['total Population']=0
    df5['total Crude Rate']=0
    
    
    vol=df['State'].unique()
    vol2=df['Gender'].unique()
    vol3=df['Race'].unique()
    vol4=df['Age Group Code'].unique()
    vol5=df['Year'].unique()
    
    for x in vol:
        for y in vol2:
            for z in vol3:
                for g in vol4:
                    for i in vol5:
                        I=df.loc[(df['State']==x) & (df['Gender']==y) & (df['Race']==z) & (df['Age Group Code']==g) & (df['Year']==i)].index
                        I2=df3.loc[(df3['State']==x) & (df3['Gender']==y) & (df3['Race']==z) & (df3['Age Group Code']==g) & (df3['Year']==i)].index
                        I3=df4.loc[(df4['State']==x) & (df4['Gender']==y) & (df4['Race']==z) & (df4['Age Group Code']==g) & (df4['Year']==i)].index
                        I4=df2.loc[(df2['State']==x) & (df2['Gender']==y) & (df2['Race']==z) & (df2['Age Group Code']==g) & (df2['Year']==i)].index
                                         
                        if len(I)>0 and len(I2)>0 and len(I3)>0 and len(I4)>0:
                            df5['cancer Deaths'].loc[I[0]]=df3['Deaths'].loc[I2[0]]
                            df5['cancer Population'].loc[I[0]]=df3['Population'].loc[I2[0]]
                            df5['cancer Crude Rate'].loc[I[0]]=df3['Crude Rate'].loc[I2[0]]
                            
                            df5['breath Deaths'].loc[I[0]]=df2['Deaths'].loc[I4[0]]
                            df5['breath Population'].loc[I[0]]=df2['Population'].loc[I4[0]]
                            df5['breath Crude Rate'].loc[I[0]]=df2['Crude Rate'].loc[I4[0]]
                            
                            df5['total Deaths'].loc[I[0]]=df4['Deaths'].loc[I3[0]]
                            df5['total Population'].loc[I[0]]=df4['Population'].loc[I3[0]]
                            df5['total Crude Rate'].loc[I[0]]=df4['Crude Rate'].loc[I3[0]]
                    
                    
    return df5

In [11]:
def indexchange(df):
    x=[]
    [x.append(i) for i in range(len(df))]
    df.index=x
    return df

## Integrating disease mortality datasets

In [12]:
df2 = pd.read_csv("Compressed Mortality, 1999-2016 (breath).csv", encoding='latin1')
df2=clean(df2)
df2

,State,Year,Race,Gender,Age Group Code,Deaths,Population,Crude Rate
0,Alabama,2011,Black or African American,Female,45-54,30,95754,31.3
1,Alabama,2011,Black or African American,Female,55-64,36,78217,46
2,Alabama,2011,Black or African American,Female,65-74,65,39574,164.2
3,Alabama,2011,Black or African American,Female,75-84,98,23761,412.4
4,Alabama,2011,Black or African American,Female,85+,79,10542,749.4
...,...,...,...,...,...,...,...,...
6129,Wyoming,2016,White,Female,85+,80,6355,1258.9
6130,Wyoming,2016,White,Male,55-64,34,39522,86
6131,Wyoming,2016,White,Male,65-74,65,26010,249.9
6132,Wyoming,2016,White,Male,75-84,88,10763,817.6


In [13]:
df3 = pd.read_csv("Compressed Mortality, 1999-2016 (cancer).csv", encoding='latin1')
df3=clean(df3)
df3

,State,Year,Race,Gender,Age Group Code,Deaths,Population,Crude Rate
0,Alabama,2011,Black or African American,Female,25-34,17,96794,17.6
1,Alabama,2011,Black or African American,Female,35-44,42,88219,47.6
2,Alabama,2011,Black or African American,Female,45-54,143,95754,149.3
3,Alabama,2011,Black or African American,Female,55-64,258,78217,329.9
4,Alabama,2011,Black or African American,Female,65-74,257,39574,649.4
...,...,...,...,...,...,...,...,...
8950,Wyoming,2016,White,Male,45-54,31,33575,92.3
8951,Wyoming,2016,White,Male,55-64,105,39522,265.7
8952,Wyoming,2016,White,Male,65-74,155,26010,595.9
8953,Wyoming,2016,White,Male,75-84,135,10763,1254.3


In [14]:
df4 = pd.read_csv("Compressed Mortality, 1999-2016.csv", encoding='latin1')
df4=clean(df4)
df4

,State,Year,Gender,Race,Age Group Code,Deaths,Population,Crude Rate
0,Alabama,2011,Female,Asian or Pacific Islander,55-64,10,3406,293.6
1,Alabama,2011,Female,Asian or Pacific Islander,65-74,13,1591,817.1
2,Alabama,2011,Female,Asian or Pacific Islander,75-84,20,725,2758.6
3,Alabama,2011,Female,Black or African American,1,88,8997,978.1
4,Alabama,2011,Female,Black or African American,1ÔÂ4ÈÕ,22,38296,57.4
...,...,...,...,...,...,...,...,...
17169,Wyoming,2016,Male,White,45-54,166,33575,494.4
17170,Wyoming,2016,Male,White,55-64,375,39522,948.8
17171,Wyoming,2016,Male,White,65-74,525,26010,2018.5
17172,Wyoming,2016,Male,White,75-84,542,10763,5035.8


In [15]:
df5 = pd.read_csv("Compressed Mortality, 1999-2016 (Heart disease caused by hypertension).csv", encoding='latin1')
df5=clean(df5)
df5

,State,Year,Race,Gender,Age Group Code,Deaths,Population,Crude Rate
0,Alabama,2011,Black or African American,Female,75-84,12,23761,50.5
1,Alabama,2011,Black or African American,Female,85+,14,10542,132.8
2,Alabama,2011,Black or African American,Male,45-54,15,79607,18.8
3,Alabama,2011,Black or African American,Male,55-64,13,64821,20.1
4,Alabama,2011,White,Female,45-54,13,249596,5.2
...,...,...,...,...,...,...,...,...
3877,Wisconsin,2016,White,Male,45-54,28,356181,7.9
3878,Wisconsin,2016,White,Male,55-64,59,373939,15.8
3879,Wisconsin,2016,White,Male,65-74,44,244115,18
3880,Wisconsin,2016,White,Male,75-84,47,113753,41.3


In [ ]:
x=df2.loc[df2['Age Group Code']=='1ÔÂ4ÈÕ'].index
df2=df2.drop(index=x)
df2=Unifiedformat(df2,'Age Group Code','1')

In [19]:
df3=Unifiedformat(df3,'Age Group Code','5ÔÂ9ÈÕ')
df3=Unifiedformat(df3,'Age Group Code','1')
df3=Unifiedformat(df3,'Age Group Code','1ÔÂ4ÈÕ')
df3=Unifiedformat(df3,'Age Group Code','10ÔÂ14ÈÕ')
df3

,State,Year,Race,Gender,Age Group Code,Deaths,Population,Crude Rate
0,Alabama,2011,Black or African American,Female,25-34,17,96794,17.6
1,Alabama,2011,Black or African American,Female,35-44,42,88219,47.6
2,Alabama,2011,Black or African American,Female,45-54,143,95754,149.3
3,Alabama,2011,Black or African American,Female,55-64,258,78217,329.9
4,Alabama,2011,Black or African American,Female,65-74,257,39574,649.4
...,...,...,...,...,...,...,...,...
8950,Wyoming,2016,White,Male,45-54,31,33575,92.3
8951,Wyoming,2016,White,Male,55-64,105,39522,265.7
8952,Wyoming,2016,White,Male,65-74,155,26010,595.9
8953,Wyoming,2016,White,Male,75-84,135,10763,1254.3


In [21]:
df4=Unifiedformat(df4,'Age Group Code','1')
df4=Unifiedformat(df4,'Age Group Code','1ÔÂ4ÈÕ')
df4=Unifiedformat(df4,'Age Group Code','10ÔÂ14ÈÕ')
df4=Unifiedformat(df4,'Age Group Code','5ÔÂ9ÈÕ')
df4=Unifiedformat(df4,'Age Group Code','NS')
df4.head()

,State,Year,Gender,Race,Age Group Code,Deaths,Population,Crude Rate
0,Alabama,2011,Female,Asian or Pacific Islander,55-64,10,3406,293.6
1,Alabama,2011,Female,Asian or Pacific Islander,65-74,13,1591,817.1
2,Alabama,2011,Female,Asian or Pacific Islander,75-84,20,725,2758.6
6,Alabama,2011,Female,Black or African American,15-19,23,54401,42.3
7,Alabama,2011,Female,Black or African American,20-24,34,56973,59.7


In [26]:
df2=indexchange(df2)
df3=indexchange(df3)
df4=indexchange(df4)
df5=indexchange(df5)

In [28]:
df6=createData2(df5,df2,df3,df4)

In [29]:
df6.head()

,State,Year,Race,Gender,Age Group Code,Deaths,Population,Crude Rate,breath Deaths,breath Population,breath Crude Rate,cancer Deaths,cancer Population,cancer Crude Rate,total Deaths,total Population,total Crude Rate
0,Alabama,2011,Black or African American,Female,75-84,12,23761,50.5,98,23761,412.4,233,23761,980.6,1258,23761,5294.4
1,Alabama,2011,Black or African American,Female,85+,14,10542,132.8,79,10542,749.4,138,10542,1309,1479,10542,14029.6
2,Alabama,2011,Black or African American,Male,45-54,15,79607,18.8,29,79607,36.4,129,79607,162,737,79607,925.8
3,Alabama,2011,Black or African American,Male,55-64,13,64821,20.1,66,64821,101.8,378,64821,583.1,1279,64821,1973.1
4,Alabama,2011,White,Female,45-54,13,249596,5.2,70,249596,28,313,249596,125.4,1101,249596,441.1


In [31]:
len(df6)

3882

In [30]:
print(len(df6.loc[df6['cancer Deaths']==0]))
df6.loc[df6['cancer Deaths']==0]

86


,State,Year,Race,Gender,Age Group Code,Deaths,Population,Crude Rate,breath Deaths,breath Population,breath Crude Rate,cancer Deaths,cancer Population,cancer Crude Rate,total Deaths,total Population,total Crude Rate
179,Arkansas,2012,Black or African American,Male,45-54,13,27645,47.0,0,0,0,0,0,0,0,0,0
193,Arkansas,2013,Black or African American,Male,45-54,13,27074,48.0,0,0,0,0,0,0,0,0,0
254,California,2011,Asian or Pacific Islander,Male,35-44,15,411497,3.6,0,0,0,0,0,0,0,0,0
301,California,2012,Black or African American,Male,35-44,19,174584,10.9,0,0,0,0,0,0,0,0,0
324,California,2013,Asian or Pacific Islander,Male,35-44,10,425383,2.4,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3687,Virginia,2015,Black or African American,Male,35-44,10,104169,9.6,0,0,0,0,0,0,0,0,0
3811,Wisconsin,2011,Black or African American,Male,45-54,12,22002,54.5,0,0,0,0,0,0,0,0,0
3822,Wisconsin,2012,Black or African American,Male,45-54,15,21857,68.6,0,0,0,0,0,0,0,0,0
3845,Wisconsin,2014,Black or African American,Male,35-44,10,24365,41.0,0,0,0,0,0,0,0,0,0


In [32]:
df6=df6.rename(columns={'Deaths': 'heart disease Deaths', 'Population': 'heart disease Population', 'Crude Rate': 'heart disease Crude Rate'})

In [33]:
df6=Unifiedformat(df6,'cancer Deaths',0)

In [35]:
print(len(df6.loc[df6['cancer Deaths']==0]))
df6.loc[df6['cancer Deaths']==0]
len(df6)

0


3796

In [36]:
df6=indexchange(df6)
df6.to_csv('4total.csv', index=False, encoding='utf-8')

In [ ]:
df6 = pd.read_csv("4total.csv", encoding='latin1')
df6.head()

## Integrated Tobacco Use dataset 

In [213]:
# Step 1
df = pd.read_csv("Behavioral_Risk_Factor_Data__Tobacco_Use__2011_to_present_.csv", encoding='latin1')
df.head()

,YEAR,LocationAbbr,LocationDesc,TopicType,TopicDesc,MeasureDesc,DataSource,Response,Data_Value_Unit,Data_Value_Type,...,GeoLocation,TopicTypeId,TopicId,MeasureId,StratificationID1,StratificationID2,StratificationID3,StratificationID4,SubMeasureID,DisplayOrder
0,2019,US,National Median (States and DC),Tobacco Use â Survey Data,Cessation (Adults),Percent of Former Smokers Among Ever Smokers,BRFSS,NaN,%,Percentage,...,NaN,BEH,101BEH,169QUP,1GEN,8AGE,6RAC,6EDU,BRF05,5
1,2019,NC,North Carolina,Tobacco Use â Survey Data,Smokeless Tobacco Use (Adults),Current Use,BRFSS,NaN,%,Percentage,...,"(35.466220975000454, -79.15925046299964)",BEH,150BEH,177SCU,1GEN,8AGE,4RAC,6EDU,BRF71,71
2,2019,TN,Tennessee,Tobacco Use â Survey Data,Smokeless Tobacco Use (Adults),Current Use,BRFSS,NaN,%,Percentage,...,"(35.68094058000048, -85.77449091399967)",BEH,150BEH,177SCU,1GEN,8AGE,1RAC,6EDU,BRF71,71
3,2018-2019,WY,Wyoming,Tobacco Use â Survey Data,Cigarette Use (Adults),Current Smoking â (2 yrs â Race/Ethnicity),BRFSS,NaN,%,Percentage,...,"(43.23554134300048, -108.10983035299967)",BEH,100BEH,112CS2,1GEN,8AGE,1RAC,6EDU,BRF30,30
4,2018-2019,WY,Wyoming,Tobacco Use â Survey Data,Cigarette Use (Adults),Current Smoking â (2 yrs â Race/Ethnicity),BRFSS,NaN,%,Percentage,...,"(43.23554134300048, -108.10983035299967)",BEH,100BEH,112CS2,1GEN,8AGE,5RAC,6EDU,BRF34,34


In [215]:
# Step 2
df7=df[['YEAR','LocationDesc','Race', 'Age', 'Data_Value_Type', 'Data_Value', 'Gender', 'Sample_Size']]

In [225]:
# Step 3
df8=df7.loc[(df7['YEAR'] == 2011) | (df7['YEAR'] == '2011') | (df7['YEAR'] == 2012) | (df7['YEAR'] == '2012')
            | (df7['YEAR'] == 2013) | (df7['YEAR'] == '2013') | (df7['YEAR'] == 2014) | (df7['YEAR'] == '2014')
            | (df7['YEAR'] == 2015) | (df7['YEAR'] == '2015') | (df7['YEAR'] == 2016) | (df7['YEAR'] == '2016')]


In [226]:
# Step 4
df8=Unifiedformat(df8,'Age','All Ages')
len(df8)

7579

In [86]:
# Step 5
df8.dropna(inplace=True)
len(df8)

7579

In [229]:
# Step 6
df8['Age'].replace('Age 25 and Older', '25-34', inplace=True)
df8['Age'].replace('18 to 44 Years', '35-44', inplace=True)
df8['Age'].replace('25 to 44 Years', '45-54', inplace=True)
df8['Age'].replace('45 to 64 Years', '55-64', inplace=True)
df8['Age'].replace('65 Years and Older', '65+', inplace=True)

In [231]:
# Step 7
df8=indexchange(df8)
df8.to_csv('Behavioral_Risk_Factor.csv', index=False, encoding='utf-8')

### Unify the field format of Age

In [ ]:
# Step 8
col=['heart disease Deaths', 'heart disease Population',
       'heart disease Crude Rate', 'breath Deaths', 'breath Population',
       'breath Crude Rate', 'cancer Deaths', 'cancer Population',
       'cancer Crude Rate', 'total Deaths', 'total Population',
       'total Crude Rate']
for i in col:
    for j in range(len(df6)):
        df6[i].iloc[j]=float(df6[i].iloc[j])

In [ ]:
# Step 9
test=df6.loc[(df6['Age Group Code'] == '65-74') | (df6['Age Group Code'] == '75-84') | (df6['Age Group Code'] == '85+')]

In [ ]:
# Step 10
grouped_df = test.groupby(['State', 'Year', 'Race', 'Gender']).agg({'heart disease Deaths': 'sum', 
                                                                    'heart disease Population': 'sum','heart disease Crude Rate': 'sum', 'breath Deaths': 'sum', 'breath Population': 'sum',
                                                                       'breath Crude Rate': 'sum', 'cancer Deaths': 'sum', 'cancer Population': 'sum',
                                                                       'cancer Crude Rate': 'sum', 'total Deaths': 'sum', 'total Population': 'sum',
                                                                       'total Crude Rate': 'sum'}).reset_index()
grouped_df.head()

In [ ]:
# Step 11
grouped_df.insert(grouped_df.columns.get_loc('Gender') + 1, 'Age Group Code', '65+')
grouped_df.head()

In [201]:
# Step 12
test2=df6.loc[(df6['Age Group Code'] != '65-74') & (df6['Age Group Code'] != '75-84') & (df6['Age Group Code'] != '85+')]

In [202]:
# Step 13
merged_df = pd.concat([test2, grouped_df], ignore_index=True)

In [203]:
merged_df

,State,Year,Race,Gender,Age Group Code,heart disease Deaths,heart disease Population,heart disease Crude Rate,breath Deaths,breath Population,breath Crude Rate,cancer Deaths,cancer Population,cancer Crude Rate,total Deaths,total Population,total Crude Rate
0,Alabama,2011,Black or African American,Male,45-54,15.0,79607.0,18.8,29.0,79607.0,36.4,129.0,79607.0,162,737.0,79607,925.8
1,Alabama,2011,Black or African American,Male,55-64,13.0,64821.0,20.1,66.0,64821.0,101.8,378.0,64821.0,583.1,1279.0,64821,1973.1
2,Alabama,2011,White,Female,45-54,13.0,249596.0,5.2,70.0,249596.0,28,313.0,249596.0,125.4,1101.0,249596,441.1
3,Alabama,2011,White,Male,45-54,23.0,247666.0,9.3,88.0,247666.0,35.5,358.0,247666.0,144.5,1687.0,247666,681.2
4,Alabama,2011,White,Male,55-64,20.0,222503.0,9,249.0,222503.0,111.9,1005.0,222503.0,451.7,3070.0,222503,1379.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,Wisconsin,2015,Black or African American,Male,65+,10.0,7608.0,131.4,18.0,7608.0,236.6,75.0,7608.0,985.8,247.0,7608,3246.6
2474,Wisconsin,2015,White,Female,65+,250.0,472251.0,253.7,2175.0,472251.0,1937.9,3859.0,472251.0,2942.3,20433.0,472251,19233.6
2475,Wisconsin,2015,White,Male,65+,158.0,388166.0,233.8,2069.0,388166.0,2863.3,4394.0,388166.0,4861.1,17614.0,388166,23855.2
2476,Wisconsin,2016,White,Female,65+,284.0,483406.0,282.8,2029.0,483406.0,1762.6,3864.0,483406.0,2912.6,20222.0,483406,18892.7


In [234]:
# Step 14
merged_df=indexchange(merged_df)
merged_df.to_csv('4total.csv', index=False, encoding='utf-8')

In [235]:
# Step 15
df6 = pd.read_csv("4total.csv", encoding='latin1')
df6.head()

,State,Year,Race,Gender,Age Group Code,heart disease Deaths,heart disease Population,heart disease Crude Rate,breath Deaths,breath Population,breath Crude Rate,cancer Deaths,cancer Population,cancer Crude Rate,total Deaths,total Population,total Crude Rate
0,Alabama,2011,Black or African American,Male,45-54,15.0,79607.0,18.8,29.0,79607.0,36.4,129.0,79607.0,162.0,737.0,79607,925.8
1,Alabama,2011,Black or African American,Male,55-64,13.0,64821.0,20.1,66.0,64821.0,101.8,378.0,64821.0,583.1,1279.0,64821,1973.1
2,Alabama,2011,White,Female,45-54,13.0,249596.0,5.2,70.0,249596.0,28.0,313.0,249596.0,125.4,1101.0,249596,441.1
3,Alabama,2011,White,Male,45-54,23.0,247666.0,9.3,88.0,247666.0,35.5,358.0,247666.0,144.5,1687.0,247666,681.2
4,Alabama,2011,White,Male,55-64,20.0,222503.0,9.0,249.0,222503.0,111.9,1005.0,222503.0,451.7,3070.0,222503,1379.8


In [262]:
# Step 16
grouped_df8 = df8.groupby(['LocationDesc', 'YEAR', 'Age']).agg({'Data_Value': 'mean'}).reset_index()
grouped_df8.head()

,LocationDesc,YEAR,Age,Data_Value
0,Alabama,2011,18 to 24 Years,19.45
1,Alabama,2011,25-34,16.60
2,Alabama,2011,35-44,12.90
3,Alabama,2011,45-54,18.25
4,Alabama,2011,55-64,15.75


In [265]:
# Step 17
grouped_df8=indexchange(grouped_df8)

In [306]:
# Step 18
df6['Tobacco Use Rate']=0.0

In [307]:
# Step 19
vol=df6['State'].unique()
vol2=df6['Year'].unique()
vol3=df6['Age Group Code'].unique()

N=0

for x in vol:
    for y in vol2:
        for z in vol3:
            I= grouped_df8.loc[(grouped_df8['LocationDesc'] == x) & (grouped_df8['YEAR'] == y) & (grouped_df8['Age'] == z)].index
            I2= df6.loc[(df6['State'] == x) & (df6['Year'] == y) & (df6['Age Group Code'] == z)].index
            #print(I)
            
            if len(I)>0 and len(I2)>0:
                result=grouped_df8['Data_Value'].loc[I[0]]
                for i in range(len(I2)):
                    df6['Tobacco Use Rate'].loc[I2[i]]=result
            else:
                N+=1
    

In [308]:
df6

,State,Year,Race,Gender,Age Group Code,heart disease Deaths,heart disease Population,heart disease Crude Rate,breath Deaths,breath Population,breath Crude Rate,cancer Deaths,cancer Population,cancer Crude Rate,total Deaths,total Population,total Crude Rate,Tobacco Use Rate
0,Alabama,2011,Black or African American,Male,45-54,15.0,79607.0,18.8,29.0,79607.0,36.4,129.0,79607.0,162.0,737.0,79607,925.8,18.25
1,Alabama,2011,Black or African American,Male,55-64,13.0,64821.0,20.1,66.0,64821.0,101.8,378.0,64821.0,583.1,1279.0,64821,1973.1,15.75
2,Alabama,2011,White,Female,45-54,13.0,249596.0,5.2,70.0,249596.0,28.0,313.0,249596.0,125.4,1101.0,249596,441.1,18.25
3,Alabama,2011,White,Male,45-54,23.0,247666.0,9.3,88.0,247666.0,35.5,358.0,247666.0,144.5,1687.0,247666,681.2,18.25
4,Alabama,2011,White,Male,55-64,20.0,222503.0,9.0,249.0,222503.0,111.9,1005.0,222503.0,451.7,3070.0,222503,1379.8,15.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1537,Wisconsin,2012,White,Male,65+,96.0,349089.0,142.1,1821.0,349089.0,2652.0,4236.0,349089.0,4942.7,16312.0,349089,23491.8,4.50
1538,Wisconsin,2013,White,Female,65+,217.0,450433.0,231.9,2231.0,450433.0,2037.0,3864.0,450433.0,3026.4,19995.0,450433,19167.6,6.40
1539,Wisconsin,2013,White,Male,65+,137.0,361789.0,209.5,1905.0,361789.0,2737.5,4259.0,361789.0,4880.8,16957.0,361789,23880.6,6.40
1540,Wisconsin,2016,White,Female,65+,284.0,483406.0,282.8,2029.0,483406.0,1762.6,3864.0,483406.0,2912.6,20222.0,483406,18892.7,1.50


In [309]:
# Step 20
df6=Unifiedformat(df6,'Tobacco Use Rate',0.0)
len(df6)

1542

In [310]:
# Step 21
df6['Tobacco Use Rate'] = df6['Tobacco Use Rate'] / 100
df6.head()

,State,Year,Race,Gender,Age Group Code,heart disease Deaths,heart disease Population,heart disease Crude Rate,breath Deaths,breath Population,breath Crude Rate,cancer Deaths,cancer Population,cancer Crude Rate,total Deaths,total Population,total Crude Rate,Tobacco Use Rate
0,Alabama,2011,Black or African American,Male,45-54,15.0,79607.0,18.8,29.0,79607.0,36.4,129.0,79607.0,162.0,737.0,79607,925.8,0.1825
1,Alabama,2011,Black or African American,Male,55-64,13.0,64821.0,20.1,66.0,64821.0,101.8,378.0,64821.0,583.1,1279.0,64821,1973.1,0.1575
2,Alabama,2011,White,Female,45-54,13.0,249596.0,5.2,70.0,249596.0,28.0,313.0,249596.0,125.4,1101.0,249596,441.1,0.1825
3,Alabama,2011,White,Male,45-54,23.0,247666.0,9.3,88.0,247666.0,35.5,358.0,247666.0,144.5,1687.0,247666,681.2,0.1825
4,Alabama,2011,White,Male,55-64,20.0,222503.0,9.0,249.0,222503.0,111.9,1005.0,222503.0,451.7,3070.0,222503,1379.8,0.1575


In [321]:
# Step 22_3
df6['Tobacco Use Rate2']=0.0

df6=indexchange(df6)

vol=df6['State'].unique()
vol2=df6['Year'].unique()
vol3=df6['Age Group Code'].unique()

vol4=df6['Gender'].unique()
vol5=df6['Race'].unique()
N=0

for x in vol:
    for y in vol2:
        for z in vol3:
            for n in vol4:
                for m in vol5:
                    I2= df6.loc[(df6['State'] == x) & (df6['Year'] == y) & (df6['Age Group Code'] == z) & (df6['Gender'] == n) & (df6['Race'] == m)].index
            
                    if len(I2)>0:
                        totalRate=df6['Tobacco Use Rate'].loc[I2[0]]
                        RSRate=RS.loc[(RS['State'] == x)][m].iloc[0]
                        MaleR=GS.loc[(GS['State'] == x)]['ratio'].iloc[0]
                        FemaleR=1-MaleR
                        if n=='Male':
                            result=totalRate*RSRate*MaleR
                            df6['Tobacco Use Rate2'].loc[I2[0]]=result
                        else:
                            result=totalRate*RSRate*FemaleR
                            df6['Tobacco Use Rate2'].loc[I2[0]]=result
                    else:
                        N+=1


In [322]:
df6

,State,Year,Race,Gender,Age Group Code,heart disease Deaths,heart disease Population,heart disease Crude Rate,breath Deaths,breath Population,breath Crude Rate,cancer Deaths,cancer Population,cancer Crude Rate,total Deaths,total Population,total Crude Rate,Tobacco Use Rate,Tobacco Use Rate2
0,Alabama,2011,Black or African American,Male,45-54,15.0,79607.0,18.8,29.0,79607.0,36.4,129.0,79607.0,162.0,737.0,79607,925.8,0.1825,0.023535
1,Alabama,2011,Black or African American,Male,55-64,13.0,64821.0,20.1,66.0,64821.0,101.8,378.0,64821.0,583.1,1279.0,64821,1973.1,0.1575,0.020311
2,Alabama,2011,White,Female,45-54,13.0,249596.0,5.2,70.0,249596.0,28.0,313.0,249596.0,125.4,1101.0,249596,441.1,0.1825,0.063442
3,Alabama,2011,White,Male,45-54,23.0,247666.0,9.3,88.0,247666.0,35.5,358.0,247666.0,144.5,1687.0,247666,681.2,0.1825,0.059746
4,Alabama,2011,White,Male,55-64,20.0,222503.0,9.0,249.0,222503.0,111.9,1005.0,222503.0,451.7,3070.0,222503,1379.8,0.1575,0.051562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1537,Wisconsin,2012,White,Male,65+,96.0,349089.0,142.1,1821.0,349089.0,2652.0,4236.0,349089.0,4942.7,16312.0,349089,23491.8,0.0450,0.019005
1538,Wisconsin,2013,White,Female,65+,217.0,450433.0,231.9,2231.0,450433.0,2037.0,3864.0,450433.0,3026.4,19995.0,450433,19167.6,0.0640,0.026922
1539,Wisconsin,2013,White,Male,65+,137.0,361789.0,209.5,1905.0,361789.0,2737.5,4259.0,361789.0,4880.8,16957.0,361789,23880.6,0.0640,0.027030
1540,Wisconsin,2016,White,Female,65+,284.0,483406.0,282.8,2029.0,483406.0,1762.6,3864.0,483406.0,2912.6,20222.0,483406,18892.7,0.0150,0.006310


In [345]:
# Step 23
df6 = df6.drop('Tobacco Use Rate', axis=1)
df6=df6.rename(columns={'Tobacco Use Rate2': 'Tobacco Use Rate'})

In [346]:
# Step 24
df6=indexchange(df6)
df6.to_csv('5total.csv', index=False, encoding='utf-8')

### Pre-processing Race Ratio by State

In [3]:
# Step 22_1
RS = pd.read_csv("race(state).csv", encoding='latin1')
col=['White', 'Black or African American', 'American Indian or Alaska Native', 'Asian or Pacific Islander']
for i in col:
    RS[i] = RS[i].replace({'%': ''}, regex=True).astype(float) / 100
RS.head()

,State,White,Black or African American,American Indian or Alaska Native,Asian or Pacific Islander,Hawaiian,Other
0,Alabama,0.6750,0.2659,0.0051,0.0139,0.04%,3.97%
1,Alaska,0.6336,0.0324,0.1456,0.0642,1.42%,11%
2,Arizona,0.7377,0.0453,0.0434,0.0333,0.20%,13.82%
3,Arkansas,0.7537,0.1520,0.0064,0.0153,0.35%,6.92%
4,California,0.5605,0.0572,0.0079,0.1483,0.38%,22.23%


### Pre-processing Gender Ratio by State

In [12]:
# Step 22_2
GS = pd.read_csv("gender(state).csv", encoding='latin1')
GS = GS.drop('Unnamed: 5', axis=1)
for i in range(len(GS)):
    GS['Population'].iloc[i]=GS['Population'].iloc[i].replace(',', '')
    GS['Male population'].iloc[i]=GS['Male population'].iloc[i].replace(',', '')
    GS['Female Population'].iloc[i]=GS['Female Population'].iloc[i].replace(',', '')
for i in range(len(GS)): 
    GS['Population'].iloc[i]=int(GS['Population'].iloc[i])
    GS['Male population'].iloc[i]=int(GS['Male population'].iloc[i])
    GS['Female Population'].iloc[i]=int(GS['Female Population'].iloc[i])

GS['ratio']=1.0
for i in range(len(GS)):
    x=GS['Population'].iloc[i]
    y=GS['Male population'].iloc[i]
    ratio=y/x
    GS['ratio'].iloc[i]=ratio

GS=indexchange(GS)
GS.to_csv('StateGenderRatio.csv', index=False, encoding='utf-8')
GS.head()

,State,Ratio,Population,Male population,Female Population,ratio
0,Alaska,110.34,732673,383188,349485,0.523
1,North Dakota,103.37,774948,399873,375075,0.516
2,Utah,101.74,3337975,1695691,1642284,0.508
3,Colorado,101.71,5812069,2940907,2871162,0.506
4,South Dakota,101.59,895376,452165,443211,0.505


## Integrate BMI Record

In [80]:
# Step 1
df = pd.read_csv("BRFSS__Table_of_Overweight_and_Obesity__BMI_.csv", encoding='latin1')
df.head()

,Year,Locationabbr,Locationdesc,Class,Topic,Question,Response,Break_Out,Break_Out_Category,Sample_Size,...,Data_Value_Footnote,DataSource,ClassId,TopicId,LocationID,BreakoutID,BreakOutCategoryID,QuestionID,ResponseID,GeoLocation
0,2019,KY,Kentucky,Overweight and Obesity (BMI),BMI Categories,Weight classification by Body Mass Index (BMI)...,Overweight (BMI 25.0-29.9),Overall,Overall,2706,...,NaN,BRFSS,CLASS14,TOPIC09,21,BO1,CAT1,_BMI5CAT,RESP040,"(37.645970271000465, -84.77497104799966)"
1,2020,AK,Alaska,Overweight and Obesity (BMI),BMI Categories,Weight classification by Body Mass Index (BMI)...,Obese (BMI 30.0 - 99.8),"$25,000-$34,999",Household Income,90,...,NaN,BRFSS,CLASS14,TOPIC09,2,INCOME3,CAT6,_BMI5CAT,RESP039,"(64.84507995700051, -147.72205903599973)"
2,2020,AK,Alaska,Overweight and Obesity (BMI),BMI Categories,Weight classification by Body Mass Index (BMI)...,Underweight (BMI 12.0-18.4),45-54,Age Group,5,...,Prevalence estimate not available if the unwei...,BRFSS,CLASS14,TOPIC09,2,AGE04,CAT3,_BMI5CAT,RESP042,"(64.84507995700051, -147.72205903599973)"
3,2020,AZ,Arizona,Overweight and Obesity (BMI),BMI Categories,Weight classification by Body Mass Index (BMI)...,Underweight (BMI 12.0-18.4),65+,Age Group,80,...,NaN,BRFSS,CLASS14,TOPIC09,4,AGE09,CAT3,_BMI5CAT,RESP042,"(34.865970280000454, -111.76381127699972)"
4,2020,AR,Arkansas,Overweight and Obesity (BMI),BMI Categories,Weight classification by Body Mass Index (BMI)...,Underweight (BMI 12.0-18.4),65+,Age Group,49,...,NaN,BRFSS,CLASS14,TOPIC09,5,AGE09,CAT3,_BMI5CAT,RESP042,"(34.74865012400045, -92.27449074299966)"


In [93]:
# Step 2
df9=df[['Year', 'Locationdesc', 'Break_Out', 'Break_Out_Category', 'Response', 'Data_value']]
df9

,Year,Locationdesc,Break_Out,Break_Out_Category,Response,Data_value
0,2019,Kentucky,Overall,Overall,Overweight (BMI 25.0-29.9),35.36
1,2020,Alaska,"$25,000-$34,999",Household Income,Obese (BMI 30.0 - 99.8),46.37
2,2020,Alaska,45-54,Age Group,Underweight (BMI 12.0-18.4),NaN
3,2020,Arizona,65+,Age Group,Underweight (BMI 12.0-18.4),2.08
4,2020,Arkansas,65+,Age Group,Underweight (BMI 12.0-18.4),1.80
...,...,...,...,...,...,...
58533,2021,Virgin Islands,"$200,000+",Household Income,Obese (BMI 30.0 - 99.8),NaN
58534,2021,Virgin Islands,"$15,000-$24,999",Household Income,Normal Weight (BMI 18.5-24.9),28.73
58535,2021,Virgin Islands,"$50,000-$99,999",Household Income,Overweight (BMI 25.0-29.9),27.39
58536,2021,Puerto Rico,25-34,Age Group,Overweight (BMI 25.0-29.9),34.03


In [94]:
# Step 3
df9.dropna(inplace=True)

In [96]:
df10=df9.loc[(df9['Year'] == 2011) | (df9['Year'] == 2012) | (df9['Year'] == 2013) | (df9['Year'] == 2014) | (df9['Year'] == 2015) | (df9['Year'] == 2016) ]

In [97]:
df10

,Year,Locationdesc,Break_Out,Break_Out_Category,Response,Data_value
10936,2013,Puerto Rico,College graduate,Education Attained,Normal Weight (BMI 18.5-24.9),32.90
10937,2012,Guam,45-54,Age Group,Obese (BMI 30.0 - 99.8),32.80
10940,2011,Guam,45-54,Age Group,Normal Weight (BMI 18.5-24.9),32.60
10941,2012,Puerto Rico,Overall,Overall,Underweight (BMI 12.0-18.4),2.10
10942,2012,Puerto Rico,"$25,000-$34,999",Household Income,Overweight (BMI 25.0-29.9),44.40
...,...,...,...,...,...,...
41546,2016,Puerto Rico,"$25,000-$34,999",Household Income,Overweight (bmi 25.0-29.9),43.11
41548,2016,Puerto Rico,Male,Gender,Obese (bmi 30.0 - 99.8),29.96
41549,2016,Puerto Rico,"$35,000-$49,999",Household Income,Obese (bmi 30.0 - 99.8),31.46
41552,2016,Guam,Hispanic,Race/Ethnicity,Obese (bmi 30.0 - 99.8),28.83


In [350]:
# Step 4
df9['Break_Out'].replace('Black, non-Hispanic', 'Black or African American', inplace=True)
df9['Break_Out'].replace('White, non-Hispanic', 'White', inplace=True)
df9['Break_Out'].replace('Asian, non-Hispanic', 'Asian or Pacific Islander', inplace=True)
df9['Break_Out'].replace('American Indian or Alaskan Native, non-Hispanic', 'American Indian or Alaska Native', inplace=True)

In [330]:
# Step 5
df9['Response'].replace('Overweight (bmi 25.0-29.9)', 'Overweight (BMI 25.0-29.9)', inplace=True)
df9['Response'].replace('Obese (bmi 30.0 - 99.8)', 'Obese (BMI 30.0 - 99.8)', inplace=True)
df9['Response'].replace('Normal Weight (bmi 18.5-24.9)', 'Normal Weight (BMI 18.5-24.9)', inplace=True)
df9['Response'].replace('Underweight (bmi 12.0-18.4)', 'Underweight (BMI 12.0-18.4)', inplace=True)

In [332]:
# Step 6
df9['Data_value'] = df9['Data_value'] / 100
df9.head()

,Year,Locationdesc,Break_Out,Break_Out_Category,Response,Data_value
0,2019,Kentucky,Overall,Overall,Overweight (BMI 25.0-29.9),0.3536
1,2020,Alaska,"$25,000-$34,999",Household Income,Obese (BMI 30.0 - 99.8),0.4637
3,2020,Arizona,65+,Age Group,Underweight (BMI 12.0-18.4),0.0208
4,2020,Arkansas,65+,Age Group,Underweight (BMI 12.0-18.4),0.0180
5,2019,Alaska,Overall,Overall,Normal Weight (BMI 18.5-24.9),0.3221


In [352]:
# Step 7
df10=df9.loc[(df9['Year'] == 2011) | (df9['Year'] == 2012) | (df9['Year'] == 2013) | (df9['Year'] == 2014) | (df9['Year'] == 2015) | (df9['Year'] == 2016) ]

In [413]:
# Step 8
df6['Overweight (BMI 25.0-29.9)']=0.0
df6['Obese (BMI 30.0 - 99.8)']=0.0
df6['Underweight (BMI 12.0-18.4)']=0.0
df6['Normal Weight (BMI 18.5-24.9)']=0.0

df6=indexchange(df6)

vol=df6['State'].unique()
vol2=df6['Year'].unique()
vol3=df6['Age Group Code'].unique()
vol4=df6['Gender'].unique()
vol5=df6['Race'].unique()
N=0
Col=['Overweight (BMI 25.0-29.9)', 'Obese (BMI 30.0 - 99.8)', 'Underweight (BMI 12.0-18.4)', 'Normal Weight (BMI 18.5-24.9)']

for x in vol:
    for y in vol2:
        for z in vol3:
            for n in vol4:
                for m in vol5:
                    I2= df6.loc[(df6['State'] == x) & (df6['Year'] == y) & (df6['Age Group Code'] == z) & (df6['Gender'] == n) & (df6['Race'] == m)].index
                    #print('Age')

                    #print(I2)
                    if len(I2)>0:
                        
                        for i in Col:
                            k1=df10.loc[(df10['Locationdesc'] == x) & (df10['Year'] == y)  & (df10['Response'] == i) & (df10['Break_Out'] == n)]['Data_value'].index
                            k2=df10.loc[(df10['Locationdesc'] == x) & (df10['Year'] == y) & (df10['Response'] == i) & (df10['Break_Out'] == z)]['Data_value'].index
                            k3=df10.loc[(df10['Locationdesc'] == x) & (df10['Year'] == y) & (df10['Response'] == i) & (df10['Break_Out'] == m)]['Data_value'].index
                            if len(k1)>0 and len(k2)>0 and len(k3)>0:
                                Gender=df10.loc[(df10['Locationdesc'] == x) & (df10['Year'] == y)  & (df10['Response'] == i) & (df10['Break_Out'] == n)]['Data_value'].iloc[0]
                                Age=df10.loc[(df10['Locationdesc'] == x) & (df10['Year'] == y) & (df10['Response'] == i) & (df10['Break_Out'] == z)]['Data_value'].iloc[0]
                                Race=df10.loc[(df10['Locationdesc'] == x) & (df10['Year'] == y) & (df10['Response'] == i) & (df10['Break_Out'] == m)]['Data_value'].iloc[0]
                                result=Gender*Age*Race
                                df6[i].loc[I2[0]]=result

                    else:
                        N+=1


In [414]:
df6

,State,Year,Race,Gender,Age Group Code,heart disease Deaths,heart disease Population,heart disease Crude Rate,breath Deaths,breath Population,...,cancer Population,cancer Crude Rate,total Deaths,total Population,total Crude Rate,Tobacco Use Rate,Overweight (BMI 25.0-29.9),Obese (BMI 30.0 - 99.8),Underweight (BMI 12.0-18.4),Normal Weight (BMI 18.5-24.9)
0,Alabama,2011,Black or African American,Male,45-54,15.0,79607.0,18.8,29.0,79607.0,...,79607.0,162.0,737.0,79607,925.8,0.023535,0.042143,0.049219,0.000000,0.019468
1,Alabama,2011,Black or African American,Male,55-64,13.0,64821.0,20.1,66.0,64821.0,...,64821.0,583.1,1279.0,64821,1973.1,0.020311,0.045665,0.047146,0.000001,0.018614
2,Alabama,2011,White,Female,45-54,13.0,249596.0,5.2,70.0,249596.0,...,249596.0,125.4,1101.0,249596,441.1,0.063442,0.039856,0.036010,0.000000,0.027951
3,Alabama,2011,White,Male,45-54,23.0,247666.0,9.3,88.0,247666.0,...,247666.0,144.5,1687.0,247666,681.2,0.059746,0.050964,0.036577,0.000000,0.022249
4,Alabama,2011,White,Male,55-64,20.0,222503.0,9.0,249.0,222503.0,...,222503.0,451.7,3070.0,222503,1379.8,0.051562,0.055222,0.035036,0.000001,0.021274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1537,Wisconsin,2012,White,Male,65+,96.0,349089.0,142.1,1821.0,349089.0,...,349089.0,4942.7,16312.0,349089,23491.8,0.019005,0.064548,0.025759,0.000000,0.023380
1538,Wisconsin,2013,White,Female,65+,217.0,450433.0,231.9,2231.0,450433.0,...,450433.0,3026.4,19995.0,450433,19167.6,0.026922,0.045626,0.027516,0.000010,0.030697
1539,Wisconsin,2013,White,Male,65+,137.0,361789.0,209.5,1905.0,361789.0,...,361789.0,4880.8,16957.0,361789,23880.6,0.027030,0.062459,0.029434,0.000005,0.020900
1540,Wisconsin,2016,White,Female,65+,284.0,483406.0,282.8,2029.0,483406.0,...,483406.0,2912.6,20222.0,483406,18892.7,0.006310,0.040659,0.032461,0.000004,0.031205


In [421]:
# Step 9
print(len(df6))
df6=Unifiedformat(df6,'Normal Weight (BMI 18.5-24.9)', 0.0)
print(len(df6))

1542
1516


In [422]:
# Step 10
df6=indexchange(df6)
df6.to_csv('6total.csv', index=False, encoding='utf-8')